In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers,optimizers,utils,models,Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gdown
import pandas as pd

# Google Drive file ID from the provided link


df1=pd.read_csv('/content/drive/MyDrive/df_indicbert.csv')
df2=pd.read_csv('/content/drive/MyDrive/df_mbert.csv')
df3=pd.read_csv('/content/drive/MyDrive/df_indicbart.csv')
df4=pd.read_csv('/content/drive/MyDrive/resnet.csv')
df5=pd.read_csv('/content/drive/MyDrive/vgg.csv')
df6=pd.read_csv('/content/drive/MyDrive/vit.csv')

In [ ]:
df5.columns

Index(['image_name', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       ...
       'feature_4086', 'feature_4087', 'feature_4088', 'feature_4089',
       'feature_4090', 'feature_4091', 'feature_4092', 'feature_4093',
       'feature_4094', 'feature_4095'],
      dtype='object', length=4097)

In [ ]:
df1.drop(columns=['Unnamed: 0'],inplace=True)
df2.drop(columns=['Unnamed: 0'],inplace=True)
df3.drop(columns=['Unnamed: 0'],inplace=True)
df4.drop(columns=['image_name'],inplace=True)
df5.drop(columns=['image_name'],inplace=True)
df6.drop(columns=['filename'],inplace=True)


In [ ]:
df6.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature758,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767
0,-0.416145,-0.276231,-0.392001,-0.606876,-0.545421,0.533438,1.032515,1.139428,-0.742312,-0.548887,...,-0.789767,0.192972,0.398747,0.078845,-0.566895,-0.497367,-0.782719,-0.003692,0.680532,-0.271811
1,-0.965692,0.412326,-0.475960,-0.357032,0.230530,0.285518,0.067791,0.186130,-0.319218,-0.473112,...,0.952189,0.238252,1.045504,-0.191923,-0.759218,0.348474,0.451487,-0.857950,-0.218055,0.335022
2,-0.629101,-1.348141,-0.807803,-0.961893,-0.812515,-0.819312,0.387613,1.059601,0.612269,-0.042704,...,-0.697618,0.204462,-0.291613,0.096057,-0.386358,-0.017975,-0.287205,-0.146279,0.501961,-0.097140
3,-0.876802,-0.648804,0.653484,-0.045389,0.659925,0.401616,0.679141,1.194811,-0.453751,-0.825877,...,-0.309081,0.202028,1.344246,0.664721,0.351493,-0.135869,-0.867777,-0.119851,0.098171,0.621293
4,-0.215062,0.360699,0.220220,-0.977994,1.553797,0.316015,0.338799,-0.137700,-0.286709,-0.299740,...,-0.046648,0.291260,0.664243,-0.076897,0.793650,0.805758,-0.311186,0.186178,-0.695267,0.654156


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/final_dataset.csv")
labels = df['Results'].replace({"सच": 0, "झूठ": 1})

In [ ]:
df1["labels"] = labels
df2["labels"] = labels
df3["labels"] = labels
df4["labels"] = labels
df5["labels"] = labels
df6["labels"] = labels

In [ ]:
def preprocess(df1,df2):
  df = pd.concat([df1,df2], axis=1)
  cols=["col"+f"{i}" for i in range (1536)]
  cols.append('labels')
  df.columns=cols
  df_1=df[df['labels']==0]
  df_2=df[df['labels']==1]
  temp=[]

  for i in range(df_1.shape[0]):
    temp.append(df_1.iloc[i])
    temp.append(df_2.iloc[i])

  updated_df=pd.DataFrame(temp)

  return updated_df

In [ ]:
def preprocess2(dataset):
  cols=["col"+f"{i}" for i in range (dataset.shape[1]-1)]
  cols.append('labels')
  dataset.columns=cols
  df_1=dataset[dataset['labels']==0]
  df_2=dataset[dataset['labels']==1]
  temp=[]

  for i in range(df_1.shape[0]):
    temp.append(df_1.iloc[i])
    temp.append(df_2.iloc[i])

  updated_df=pd.DataFrame(temp)

  return updated_df

# **Fusion**

In [ ]:
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Concatenate, Reshape
from tensorflow.keras.models import Model

def early_fusion1(img_arr, text_arr, labels):
    input_layer1 = Input(shape=(img_arr.shape[1],))
    input_layer2 = Input(shape=(text_arr.shape[1],))

    reshape_layer1 = Reshape((1, img_arr.shape[1]))(input_layer1)
    reshape_layer2 = Reshape((1, text_arr.shape[1]))(input_layer2)

    bilstm_output1 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer1)
    bilstm_output2 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer2)

    concatenated_features = Concatenate(axis=1)([bilstm_output1, bilstm_output2])

    dense_layer = Dense(512, activation='relu')(concatenated_features)
    dense_layer = Dense(256, activation='relu')(dense_layer)
    dense_layer = Dense(128, activation='relu')(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)

    output_layer = Dense(1, activation='sigmoid', name='class')(dense_layer)

    model = Model(inputs=[input_layer1, input_layer2], outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit([img_arr[0:3926], text_arr[0:3926]], labels[0:3926], epochs=10, batch_size=32)

    return model



In [ ]:
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Concatenate, Reshape
from tensorflow.keras.models import Model

def early_fusion2(img_arr, text_arr, labels):
    input_layer1 = Input(shape=(img_arr.shape[1],))
    input_layer2 = Input(shape=(text_arr.shape[1],))

    reshape_layer1 = Reshape((1, img_arr.shape[1]))(input_layer1)
    reshape_layer2 = Reshape((1, text_arr.shape[1]))(input_layer2)

    bilstm_output1 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer1)
    bilstm_output2 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer2)

    multiplied_features = tf.math.multiply(bilstm_output1,bilstm_output2)

    sign_of_train_features = tf.math.sign(multiplied_features)
    abs_of_train_features = tf.math.abs(multiplied_features)

    sqrt_of_train_features = tf.math.sqrt(abs_of_train_features)
    power_normalized_train_features = tf.math.multiply(sign_of_train_features,sqrt_of_train_features)
    l2_normalized_features = tf.math.l2_normalize(power_normalized_train_features, axis = 0)
    dense_layer = Dense(512,activation = 'relu')(l2_normalized_features)
    dense_layer = Dense(256, activation='relu')(dense_layer)
    dense_layer = Dense(128, activation='relu')(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)

    output_layer = Dense(1, activation='sigmoid', name='class')(dense_layer)

    model = Model(inputs=[input_layer1, input_layer2], outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit([img_arr[0:3926], text_arr[0:3926]], labels[0:3926], epochs=10, batch_size=32)

    return model



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def eval1(model, img_arr, text_arr, labels):
    # Reshape test data to match the expected input shape of the model
    text_test_data=text_arr[3926:]
    img_test_data=img_arr[3926:]
    test_labels=labels[3926:]
    text_test_data_reshaped = text_test_data.reshape(-1, text_test_data.shape[1])
    img_test_data_reshaped = img_test_data.reshape(-1, img_test_data.shape[1])

    # Combine test data into a single input for the model
    test_data = [img_test_data_reshaped, text_test_data_reshaped]

    # Make predictions using the trained model
    predictions = model.predict(test_data)
    binary_predictions = (predictions >= 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(test_labels, binary_predictions)
    precision = precision_score(test_labels, binary_predictions)
    recall = recall_score(test_labels, binary_predictions)
    f1 = f1_score(test_labels, binary_predictions)
    conf_matrix = confusion_matrix(test_labels, binary_predictions)

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("Confusion Matrix:\n", conf_matrix)


In [ ]:
def late_fusion(img_arr, text_arr, labels):

    input_layer1 = Input(shape=(img_arr.shape[1],))
    reshape_layer1 = Reshape((1, img_arr.shape[1]))(input_layer1)
    bilstm_output1 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer1)

    dense_layer = Dense(512, activation='relu')(bilstm_output1)
    dense_layer = Dense(256, activation='relu')(dense_layer)
    dense_layer = Dense(128, activation='relu')(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)

    output_layer = Dense(1, activation='sigmoid', name='class')(dense_layer)

    model1 = Model(inputs=input_layer1, outputs=output_layer)
    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model1.fit(img_arr[0:3926], labels[0:3926], epochs=10, batch_size=32)





    input_layer2 = Input(shape=(text_arr.shape[1],))
    reshape_layer2 = Reshape((1, text_arr.shape[1]))(input_layer2)
    bilstm_output2 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer2)

    dense_layer = Dense(512, activation='relu')(bilstm_output2)
    dense_layer = Dense(256, activation='relu')(dense_layer)
    dense_layer = Dense(128, activation='relu')(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)

    output_layer = Dense(1, activation='sigmoid', name='class')(dense_layer)

    model2 = Model(inputs=input_layer2, outputs=output_layer)
    model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model2.fit(text_arr[0:3926], labels[0:3926], epochs=10, batch_size=32)

    return [model1,model2]

In [ ]:
def eval2(model1,model2, img_arr, text_arr, labels):
    # Reshape test data to match the expected input shape of the model
    text_test_data=text_arr[3926:]
    img_test_data=img_arr[3926:]
    test_labels=labels[3926:]
    text_test_data_reshaped = text_test_data.reshape(-1, text_test_data.shape[1])
    img_test_data_reshaped = img_test_data.reshape(-1, img_test_data.shape[1])

    prediction1=model1.predict(img_test_data)
    prediction2 = model2.predict(text_test_data)
    predictions=(prediction1+prediction2)/2.0
    binary_predictions = (predictions >= 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(test_labels, binary_predictions)
    precision = precision_score(test_labels, binary_predictions)
    recall = recall_score(test_labels, binary_predictions)
    f1 = f1_score(test_labels, binary_predictions)
    conf_matrix = confusion_matrix(test_labels, binary_predictions)

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("Confusion Matrix:\n", conf_matrix)

In [ ]:
final_df1=preprocess(df1,df4)
final_df2=preprocess(df1,df5)
final_df3=preprocess(df1,df6)

final_df4=preprocess(df2,df4)
final_df5=preprocess(df2,df5)
final_df6=preprocess(df2,df6)

final_df7=preprocess(df3,df4)
final_df8=preprocess(df3,df5)
final_df9=preprocess(df3,df6)


text_arr_final_df1=final_df1.iloc[:,0:768].values
img_arr_final_df1=final_df1.iloc[:,768:1536].values
labels_final_df1=final_df1['labels'].values

text_arr_final_df2=final_df2.iloc[:,0:768].values
img_arr_final_df2=final_df2.iloc[:,768:1536].values
labels_final_df2=final_df2['labels'].values

text_arr_final_df3=final_df3.iloc[:,0:768].values
img_arr_final_df3=final_df3.iloc[:,768:1536].values
labels_final_df3=final_df3['labels'].values

text_arr_final_df4=final_df4.iloc[:,0:768].values
img_arr_final_df4=final_df4.iloc[:,768:1536].values
labels_final_df4=final_df4['labels'].values

text_arr_final_df5=final_df5.iloc[:,0:768].values
img_arr_final_df5=final_df5.iloc[:,768:1536].values
labels_final_df5=final_df5['labels'].values

text_arr_final_df6=final_df6.iloc[:,0:768].values
img_arr_final_df6=final_df6.iloc[:,768:1536].values
labels_final_df6=final_df6['labels'].values

text_arr_final_df7=final_df7.iloc[:,0:768].values
img_arr_final_df7=final_df7.iloc[:,768:1536].values
labels_final_df7=final_df7['labels'].values

text_arr_final_df8=final_df8.iloc[:,0:768].values
img_arr_final_df8=final_df8.iloc[:,768:1536].values
labels_final_df8=final_df8['labels'].values

text_arr_final_df9=final_df9.iloc[:,0:768].values
img_arr_final_df9=final_df9.iloc[:,768:1536].values
labels_final_df9=final_df9['labels'].values

In [ ]:
# import gdown
# import pandas as pd

# # Google Drive file ID from the provided link


# df1=pd.read_csv('/content/drive/MyDrive/df_indicbert.csv')
# df2=pd.read_csv('/content/drive/MyDrive/df_mbert.csv')
# df3=pd.read_csv('/content/drive/MyDrive/df_indicbart.csv')
# df4=pd.read_csv('/content/drive/MyDrive/resnet.csv')
# df5=pd.read_csv('/content/drive/MyDrive/vgg.csv')
# df6=pd.read_csv('/content/drive/MyDrive/vit.csv')

# **Early Fusion**

In [ ]:
model=early_fusion1(img_arr_final_df1,text_arr_final_df1,labels_final_df1)
print("Indic_bert+resnet")
eval1(model,img_arr_final_df1,text_arr_final_df1,labels_final_df1)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


model=early_fusion1(img_arr_final_df2,text_arr_final_df2,labels_final_df1)
print("Indic_bert+vgg")
eval1(model,img_arr_final_df2,text_arr_final_df2,labels_final_df2)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df3,text_arr_final_df3,labels_final_df3)
print("Indic_bert+vit")
eval1(model,img_arr_final_df3,text_arr_final_df3,labels_final_df3)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df4,text_arr_final_df4,labels_final_df4)
print("mbert+resnet")
eval1(model,img_arr_final_df4,text_arr_final_df4,labels_final_df4)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df5,text_arr_final_df5,labels_final_df5)
print("mbert+vgg")
eval1(model,img_arr_final_df5,text_arr_final_df5,labels_final_df5)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df6,text_arr_final_df6,labels_final_df6)
print("mbert+vit")
eval1(model,img_arr_final_df6,text_arr_final_df6,labels_final_df6)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df7,text_arr_final_df7,labels_final_df7)
print("Indic_bart+resnet")
eval1(model,img_arr_final_df7,text_arr_final_df7,labels_final_df7)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df8,text_arr_final_df8,labels_final_df8)
print("Indic_bart+vgg")
eval1(model,img_arr_final_df8,text_arr_final_df8,labels_final_df8)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion1(img_arr_final_df9,text_arr_final_df9,labels_final_df9)
print("Indic_bart+vit")
eval1(model,img_arr_final_df9,text_arr_final_df9,labels_final_df9)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


Epoch 1/10
123/123 [==============================] - 17s 22ms/step - loss: 0.3563 - accuracy: 0.8441
Epoch 2/10
123/123 [==============================] - 3s 21ms/step - loss: 0.1336 - accuracy: 0.9521
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0815 - accuracy: 0.9707
Epoch 4/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0352 - accuracy: 0.9896
Epoch 5/10
123/123 [==============================] - 4s 32ms/step - loss: 0.0272 - accuracy: 0.9908
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0145 - accuracy: 0.9954
Epoch 7/10
123/123 [==============================] - 3s 23ms/step - loss: 0.0129 - accuracy: 0.9954
Epoch 8/10
123/123 [==============================] - 3s 25ms/step - loss: 0.0261 - accuracy: 0.9924
Epoch 9/10
123/123 [==============================] - 4s 31ms/step - loss: 0.0131 - accuracy: 0.9969
Epoch 10/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0139 - accur

Accuracy: 0.9106187929717342
Precision: 0.9020194465220643
Recall: 0.9213139801375095
F1-score: 0.9115646258503403
Confusion Matrix:
 [[1178  131]
 [ 103 1206]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 14s 21ms/step - loss: 0.3926 - accuracy: 0.8143
Epoch 2/10
123/123 [==============================] - 4s 36ms/step - loss: 0.1255 - accuracy: 0.9521
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0347 - accuracy: 0.9893
Epoch 4/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0310 - accuracy: 0.9890
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0258 - accuracy: 0.9908
Epoch 6/10
123/123 [==============================] - 3s 25ms/step - loss: 0.0187 - accuracy: 0.9941
Epoch 7/10
123/123 [==============================] - 4s 30ms/step - loss: 0.0174 - accuracy: 0.9944
Epoch 8/10
123/123 [==============================] - 2s 20ms/step - loss: 0.0162 - accuracy: 0.9952
Epoch 9/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0136 - accuracy: 0.9941
Epoch 10/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0131 - accur

Accuracy: 0.887318563789152
Precision: 0.8772321428571429
Recall: 0.9006875477463713
F1-score: 0.8888051262721448
Confusion Matrix:
 [[1144  165]
 [ 130 1179]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 14s 21ms/step - loss: 0.3637 - accuracy: 0.8265
Epoch 2/10
123/123 [==============================] - 4s 34ms/step - loss: 0.1918 - accuracy: 0.9215
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.1270 - accuracy: 0.9501
Epoch 4/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0793 - accuracy: 0.9689
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0451 - accuracy: 0.9817
Epoch 6/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0855 - accuracy: 0.9664
Epoch 7/10
123/123 [==============================] - 4s 36ms/step - loss: 0.0285 - accuracy: 0.9906
Epoch 8/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0213 - accuracy: 0.9926
Epoch 9/10
123/123 [==============================] - 3s 27ms/step - loss: 0.0199 - accuracy: 0.9926
Epoch 10/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0026 - accur

...

Epoch 1/10
123/123 [==============================] - 14s 26ms/step - loss: 0.2740 - accuracy: 0.8770
Epoch 2/10
123/123 [==============================] - 3s 20ms/step - loss: 0.1068 - accuracy: 0.9608
Epoch 3/10
123/123 [==============================] - 3s 20ms/step - loss: 0.0813 - accuracy: 0.9694
Epoch 4/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0466 - accuracy: 0.9845
Epoch 5/10
123/123 [==============================] - 4s 33ms/step - loss: 0.0404 - accuracy: 0.9857
Epoch 6/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0132 - accuracy: 0.9959
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0288 - accuracy: 0.9903
Epoch 8/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0226 - accuracy: 0.9934
Epoch 9/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0138 - accuracy: 0.9944
Epoch 10/10
123/123 [==============================] - 4s 36ms/step - loss: 0.0195 - accur

Epoch 1/10
123/123 [==============================] - 15s 30ms/step - loss: 0.2935 - accuracy: 0.8767
Epoch 2/10
123/123 [==============================] - 3s 28ms/step - loss: 0.0444 - accuracy: 0.9837
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0235 - accuracy: 0.9921
Epoch 4/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0218 - accuracy: 0.9939
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0110 - accuracy: 0.9980
Epoch 6/10
123/123 [==============================] - 4s 33ms/step - loss: 0.0104 - accuracy: 0.9969
Epoch 7/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0092 - accuracy: 0.9972
Epoch 8/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0038 - accuracy: 0.9987
Epoch 9/10
123/123 [==============================] - 3s 20ms/step - loss: 0.0104 - accuracy: 0.9975
Epoch 10/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0169 - accur

Accuracy: 0.9633307868601986
Precision: 0.9809674861221253
Recall: 0.944996180290298
F1-score: 0.9626459143968872
Confusion Matrix:
 [[1285   24]
 [  72 1237]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 14s 34ms/step - loss: 0.3417 - accuracy: 0.8283
Epoch 2/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0471 - accuracy: 0.9857
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0236 - accuracy: 0.9921
Epoch 4/10
123/123 [==============================] - 3s 20ms/step - loss: 0.0142 - accuracy: 0.9964
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0073 - accuracy: 0.9985
Epoch 6/10
123/123 [==============================] - 4s 34ms/step - loss: 0.0117 - accuracy: 0.9962
Epoch 7/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0141 - accuracy: 0.9954
Epoch 8/10
123/123 [==============================] - 3s 20ms/step - loss: 0.0045 - accuracy: 0.9987
Epoch 9/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0050 - accuracy: 0.9985
Epoch 10/10
123/123 [==============================] - 3s 25ms/step - loss: 6.1706e-04 - a

Epoch 1/10
123/123 [==============================] - 15s 25ms/step - loss: 0.3207 - accuracy: 0.8571
Epoch 2/10
123/123 [==============================] - 3s 20ms/step - loss: 0.0599 - accuracy: 0.9804
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0291 - accuracy: 0.9898
Epoch 4/10
123/123 [==============================] - 3s 20ms/step - loss: 0.0215 - accuracy: 0.9926
Epoch 5/10
123/123 [==============================] - 4s 35ms/step - loss: 0.0166 - accuracy: 0.9949
Epoch 6/10
123/123 [==============================] - 4s 34ms/step - loss: 0.0192 - accuracy: 0.9957
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0205 - accuracy: 0.9941
Epoch 8/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0069 - accuracy: 0.9977
Epoch 9/10
123/123 [==============================] - 3s 23ms/step - loss: 0.0090 - accuracy: 0.9980
Epoch 10/10
123/123 [==============================] - 4s 32ms/step - loss: 0.0113 - accur

# **early fusion 2**

In [ ]:
model=early_fusion2(img_arr_final_df1,text_arr_final_df1,labels_final_df1)
print("Indic_bert+resnet")
eval1(model,img_arr_final_df1,text_arr_final_df1,labels_final_df1)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


model=early_fusion2(img_arr_final_df2,text_arr_final_df2,labels_final_df1)
print("Indic_bert+vgg")
eval1(model,img_arr_final_df2,text_arr_final_df2,labels_final_df2)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df3,text_arr_final_df3,labels_final_df3)
print("Indic_bert+vit")
eval1(model,img_arr_final_df3,text_arr_final_df3,labels_final_df3)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df4,text_arr_final_df4,labels_final_df4)
print("mbert+resnet")
eval1(model,img_arr_final_df4,text_arr_final_df4,labels_final_df4)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df5,text_arr_final_df5,labels_final_df5)
print("mbert+vgg")
eval1(model,img_arr_final_df5,text_arr_final_df5,labels_final_df5)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df6,text_arr_final_df6,labels_final_df6)
print("mbert+vit")
eval1(model,img_arr_final_df6,text_arr_final_df6,labels_final_df6)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df7,text_arr_final_df7,labels_final_df7)
print("Indic_bart+resnet")
eval1(model,img_arr_final_df7,text_arr_final_df7,labels_final_df7)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df8,text_arr_final_df8,labels_final_df8)
print("Indic_bart+vgg")
eval1(model,img_arr_final_df8,text_arr_final_df8,labels_final_df8)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=early_fusion2(img_arr_final_df9,text_arr_final_df9,labels_final_df9)
print("Indic_bart+vit")
eval1(model,img_arr_final_df9,text_arr_final_df9,labels_final_df9)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


Epoch 1/10
123/123 [==============================] - 15s 21ms/step - loss: 0.4085 - accuracy: 0.8062
Epoch 2/10
123/123 [==============================] - 3s 21ms/step - loss: 0.2181 - accuracy: 0.9106
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0971 - accuracy: 0.9651
Epoch 4/10
123/123 [==============================] - 3s 27ms/step - loss: 0.0679 - accuracy: 0.9750
Epoch 5/10
123/123 [==============================] - 4s 30ms/step - loss: 0.0325 - accuracy: 0.9888
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0211 - accuracy: 0.9931
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0274 - accuracy: 0.9916
Epoch 8/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0236 - accuracy: 0.9911
Epoch 9/10
123/123 [==============================] - 4s 31ms/step - loss: 0.0242 - accuracy: 0.9924
Epoch 10/10
123/123 [==============================] - 3s 25ms/step - loss: 0.0161 - accur

Accuracy: 0.8552330022918259
Precision: 0.8587962962962963
Recall: 0.8502673796791443
F1-score: 0.854510556621881
Confusion Matrix:
 [[1126  183]
 [ 196 1113]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 14s 35ms/step - loss: 0.4678 - accuracy: 0.7680
Epoch 2/10
123/123 [==============================] - 4s 35ms/step - loss: 0.2006 - accuracy: 0.9187
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0777 - accuracy: 0.9705
Epoch 4/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0450 - accuracy: 0.9842
Epoch 5/10
123/123 [==============================] - 4s 34ms/step - loss: 0.0310 - accuracy: 0.9893
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0257 - accuracy: 0.9918
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0189 - accuracy: 0.9947
Epoch 8/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0140 - accuracy: 0.9944
Epoch 9/10
123/123 [==============================] - 3s 26ms/step - loss: 0.0260 - accuracy: 0.9913
Epoch 10/10
123/123 [==============================] - 4s 28ms/step - loss: 0.0163 - accur

Accuracy: 0.813598166539343
Precision: 0.8511548331907614
Recall: 0.760122230710466
F1-score: 0.8030669895076675
Confusion Matrix:
 [[1135  174]
 [ 314  995]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 15s 22ms/step - loss: 0.3819 - accuracy: 0.8260
Epoch 2/10
123/123 [==============================] - 3s 21ms/step - loss: 0.2089 - accuracy: 0.9162
Epoch 3/10
123/123 [==============================] - 3s 22ms/step - loss: 0.1290 - accuracy: 0.9503
Epoch 4/10
123/123 [==============================] - 4s 33ms/step - loss: 0.0824 - accuracy: 0.9702
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0696 - accuracy: 0.9748
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0337 - accuracy: 0.9888
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0567 - accuracy: 0.9773
Epoch 8/10
123/123 [==============================] - 3s 27ms/step - loss: 0.0228 - accuracy: 0.9916
Epoch 9/10
123/123 [==============================] - 4s 29ms/step - loss: 0.0167 - accuracy: 0.9931
Epoch 10/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0109 - accur

...

Epoch 1/10
123/123 [==============================] - 14s 21ms/step - loss: 0.3715 - accuracy: 0.8278
Epoch 2/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0512 - accuracy: 0.9842
Epoch 3/10
123/123 [==============================] - 4s 31ms/step - loss: 0.0147 - accuracy: 0.9959
Epoch 4/10
123/123 [==============================] - 5s 40ms/step - loss: 0.0123 - accuracy: 0.9964
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0188 - accuracy: 0.9947
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0135 - accuracy: 0.9967
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0159 - accuracy: 0.9952
Epoch 8/10
123/123 [==============================] - 5s 37ms/step - loss: 0.0047 - accuracy: 0.9985
Epoch 9/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0142 - accuracy: 0.9957
Epoch 10/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0014 - accur

Accuracy: 0.9369747899159664
Precision: 0.9554140127388535
Recall: 0.9167303284950343
F1-score: 0.935672514619883
Confusion Matrix:
 [[1253   56]
 [ 109 1200]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 14s 21ms/step - loss: 0.4110 - accuracy: 0.8082
Epoch 2/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0477 - accuracy: 0.9860
Epoch 3/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0144 - accuracy: 0.9954
Epoch 4/10
123/123 [==============================] - 6s 45ms/step - loss: 0.0194 - accuracy: 0.9939
Epoch 5/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0079 - accuracy: 0.9964
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0176 - accuracy: 0.9939
Epoch 7/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0114 - accuracy: 0.9964
Epoch 8/10
123/123 [==============================] - 3s 27ms/step - loss: 0.0114 - accuracy: 0.9977
Epoch 9/10
123/123 [==============================] - 4s 29ms/step - loss: 0.0066 - accuracy: 0.9982
Epoch 10/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0092 - accur

Accuracy: 0.9236058059587471
Precision: 0.9526530612244898
Recall: 0.891520244461421
F1-score: 0.9210734017363851
Confusion Matrix:
 [[1251   58]
 [ 142 1167]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------


Epoch 1/10
123/123 [==============================] - 14s 21ms/step - loss: 0.1959 - accuracy: 0.9187
Epoch 2/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0296 - accuracy: 0.9903
Epoch 3/10
123/123 [==============================] - 4s 32ms/step - loss: 0.0224 - accuracy: 0.9903
Epoch 4/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0247 - accuracy: 0.9906
Epoch 5/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0185 - accuracy: 0.9947
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0108 - accuracy: 0.9972
Epoch 7/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0127 - accuracy: 0.9967
Epoch 8/10
123/123 [==============================] - 4s 35ms/step - loss: 0.0132 - accuracy: 0.9962
Epoch 9/10
123/123 [==============================] - 3s 22ms/step - loss: 0.0046 - accuracy: 0.9985
Epoch 10/10
123/123 [==============================] - 3s 21ms/step - loss: 0.0105 - accur

# **late fusion**

In [ ]:
model1,model2=late_fusion(img_arr_final_df1,text_arr_final_df1,labels_final_df1)
print("Indic_bert+resnet")
eval2(model1,model2,img_arr_final_df1,text_arr_final_df1,labels_final_df1)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


model1,model2=late_fusion(img_arr_final_df2,text_arr_final_df2,labels_final_df1)
print("Indic_bert+vgg")
eval2(model1,model2,img_arr_final_df2,text_arr_final_df2,labels_final_df2)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df3,text_arr_final_df3,labels_final_df3)
print("Indic_bert+vit")
eval1(model,img_arr_final_df3,text_arr_final_df3,labels_final_df3)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df4,text_arr_final_df4,labels_final_df4)
print("mbert+resnet")
eval2(model1,model2,img_arr_final_df4,text_arr_final_df4,labels_final_df4)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df5,text_arr_final_df5,labels_final_df5)
print("mbert+vgg")
eval2(model1,model2,img_arr_final_df5,text_arr_final_df5,labels_final_df5)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df6,text_arr_final_df6,labels_final_df6)
print("mbert+vit")
eval2(model1,model2,img_arr_final_df6,text_arr_final_df6,labels_final_df6)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df7,text_arr_final_df7,labels_final_df7)
print("Indic_bart+resnet")
eval2(model1,model2,img_arr_final_df7,text_arr_final_df7,labels_final_df7)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df8,text_arr_final_df8,labels_final_df8)
print("Indic_bart+vgg")
eval2(model1,model2,img_arr_final_df8,text_arr_final_df8,labels_final_df8)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model1,model2=late_fusion(img_arr_final_df9,text_arr_final_df9,labels_final_df9)
print("Indic_bart+vit")
eval2(model1,model2,img_arr_final_df9,text_arr_final_df9,labels_final_df9)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


Epoch 1/10
123/123 [==============================] - 9s 13ms/step - loss: 0.4107 - accuracy: 0.8128
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.1868 - accuracy: 0.9238
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0898 - accuracy: 0.9702
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0542 - accuracy: 0.9804
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0278 - accuracy: 0.9901
Epoch 6/10
123/123 [==============================] - 2s 16ms/step - loss: 0.0251 - accuracy: 0.9911
Epoch 7/10
123/123 [==============================] - 3s 23ms/step - loss: 0.0227 - accuracy: 0.9931
Epoch 8/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0179 - accuracy: 0.9939
Epoch 9/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0108 - accuracy: 0.9969
Epoch 10/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0132 - accura

Epoch 1/10
123/123 [==============================] - 9s 20ms/step - loss: 0.4700 - accuracy: 0.7626
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2955 - accuracy: 0.8788
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2806 - accuracy: 0.8869
Epoch 4/10
123/123 [==============================] - 2s 12ms/step - loss: 0.2162 - accuracy: 0.9111
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2173 - accuracy: 0.9126
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2014 - accuracy: 0.9195
Epoch 7/10
123/123 [==============================] - 2s 13ms/step - loss: 0.1954 - accuracy: 0.9233
Epoch 8/10
123/123 [==============================] - 3s 24ms/step - loss: 0.1822 - accuracy: 0.9292
Epoch 9/10
123/123 [==============================] - 3s 27ms/step - loss: 0.1844 - accuracy: 0.9241
Epoch 10/10
123/123 [==============================] - 2s 17ms/step - loss: 0.1783 - accura

Accuracy: 0.8903743315508021
Precision: 0.9068471337579618
Recall: 0.8701298701298701
F1-score: 0.8881091617933723
Confusion Matrix:
 [[1192  117]
 [ 170 1139]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 9s 23ms/step - loss: 0.4612 - accuracy: 0.7720
Epoch 2/10
123/123 [==============================] - 2s 15ms/step - loss: 0.1723 - accuracy: 0.9310
Epoch 3/10
123/123 [==============================] - 2s 14ms/step - loss: 0.0655 - accuracy: 0.9768
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0273 - accuracy: 0.9888
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0293 - accuracy: 0.9888
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0339 - accuracy: 0.9903
Epoch 7/10
123/123 [===========================

Epoch 1/10
123/123 [==============================] - 8s 17ms/step - loss: 0.4733 - accuracy: 0.7685
Epoch 2/10
123/123 [==============================] - 3s 22ms/step - loss: 0.2848 - accuracy: 0.8859
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2496 - accuracy: 0.8994
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2296 - accuracy: 0.9075
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2187 - accuracy: 0.9131
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.1939 - accuracy: 0.9215
Epoch 7/10
123/123 [==============================] - 2s 13ms/step - loss: 0.1838 - accuracy: 0.9277
Epoch 8/10
123/123 [==============================] - 2s 13ms/step - loss: 0.1876 - accuracy: 0.9274
Epoch 9/10
123/123 [==============================] - 3s 22ms/step - loss: 0.1888 - accuracy: 0.9228
Epoch 10/10
123/123 [==============================] - 2s 19ms/step - loss: 0.1576 - accura

Accuracy: 0.8880825057295646
Precision: 0.8675832127351665
Recall: 0.9159663865546218
F1-score: 0.8911185432924563
Confusion Matrix:
 [[1126  183]
 [ 110 1199]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 8s 13ms/step - loss: 0.4265 - accuracy: 0.8031
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2968 - accuracy: 0.8752
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2115 - accuracy: 0.9167
Epoch 4/10
123/123 [==============================] - 3s 21ms/step - loss: 0.1282 - accuracy: 0.9496
Epoch 5/10
123/123 [==============================] - 2s 19ms/step - loss: 0.0848 - accuracy: 0.9705
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0565 - accuracy: 0.9791
Epoch 7/10
123/123 [===========================

...

Epoch 1/10
123/123 [==============================] - 11s 22ms/step - loss: 0.2314 - accuracy: 0.9244
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0737 - accuracy: 0.9763
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0569 - accuracy: 0.9799
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0568 - accuracy: 0.9817
Epoch 5/10
123/123 [==============================] - 2s 12ms/step - loss: 0.0389 - accuracy: 0.9875
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0299 - accuracy: 0.9896
Epoch 7/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0292 - accuracy: 0.9903
Epoch 8/10
123/123 [==============================] - 3s 23ms/step - loss: 0.0384 - accuracy: 0.9865
Epoch 9/10
123/123 [==============================] - 2s 19ms/step - loss: 0.0346 - accuracy: 0.9875
Epoch 10/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0286 - accur

Accuracy: 0.944996180290298
Precision: 0.9648842777334398
Recall: 0.9236058059587471
F1-score: 0.9437939110070257
Confusion Matrix:
 [[1265   44]
 [ 100 1209]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 9s 23ms/step - loss: 0.4652 - accuracy: 0.7667
Epoch 2/10
123/123 [==============================] - 2s 15ms/step - loss: 0.1745 - accuracy: 0.9307
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0642 - accuracy: 0.9776
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0408 - accuracy: 0.9865
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0291 - accuracy: 0.9906
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0219 - accuracy: 0.9924
Epoch 7/10
123/123 [============================

Epoch 1/10
123/123 [==============================] - 8s 13ms/step - loss: 0.2694 - accuracy: 0.8923
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0721 - accuracy: 0.9783
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0490 - accuracy: 0.9827
Epoch 4/10
123/123 [==============================] - 2s 15ms/step - loss: 0.0457 - accuracy: 0.9829
Epoch 5/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0412 - accuracy: 0.9847
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0436 - accuracy: 0.9842
Epoch 7/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0315 - accuracy: 0.9883
Epoch 8/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0308 - accuracy: 0.9896
Epoch 9/10
123/123 [==============================] - 2s 12ms/step - loss: 0.0297 - accuracy: 0.9901
Epoch 10/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0331 - accura

Accuracy: 0.932009167303285
Precision: 0.9498806682577565
Recall: 0.9121466768525592
F1-score: 0.9306313328137179
Confusion Matrix:
 [[1246   63]
 [ 115 1194]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 8s 13ms/step - loss: 0.4191 - accuracy: 0.8039
Epoch 2/10
123/123 [==============================] - 2s 12ms/step - loss: 0.2918 - accuracy: 0.8785
Epoch 3/10
123/123 [==============================] - 2s 15ms/step - loss: 0.2056 - accuracy: 0.9175
Epoch 4/10
123/123 [==============================] - 3s 24ms/step - loss: 0.1299 - accuracy: 0.9480
Epoch 5/10
123/123 [==============================] - 2s 14ms/step - loss: 0.0889 - accuracy: 0.9697
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0561 - accuracy: 0.9822
Epoch 7/10
123/123 [============================

Epoch 1/10
123/123 [==============================] - 8s 13ms/step - loss: 0.2412 - accuracy: 0.9050
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0826 - accuracy: 0.9735
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0598 - accuracy: 0.9801
Epoch 4/10
123/123 [==============================] - 2s 16ms/step - loss: 0.0463 - accuracy: 0.9837
Epoch 5/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0445 - accuracy: 0.9829
Epoch 6/10
123/123 [==============================] - 2s 14ms/step - loss: 0.0508 - accuracy: 0.9850
Epoch 7/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0331 - accuracy: 0.9880
Epoch 8/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0314 - accuracy: 0.9875
Epoch 9/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0251 - accuracy: 0.9903
Epoch 10/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0210 - accura

# **TEXT_ONLY**

In [ ]:
final_df1=preprocess2(df1)
final_df2=preprocess2(df2)
final_df3=preprocess2(df3)

final_df4=preprocess2(df4)
final_df5=preprocess2(df5)
final_df6=preprocess2(df6)



labels_final_df1=final_df1['labels'].values
labels_final_df2=final_df2['labels'].values
labels_final_df3=final_df3['labels'].values
labels_final_df4=final_df4['labels'].values
labels_final_df5=final_df5['labels'].values
labels_final_df6=final_df6['labels'].values


text_arr_final_df1=final_df1.iloc[:,:].values
text_arr_final_df2=final_df2.iloc[:,:].values
text_arr_final_df3=final_df3.iloc[:,:].values


img_arr_final_df1=final_df4.iloc[:,:].values
img_arr_final_df2=final_df5.iloc[:,:].values
img_arr_final_df3=final_df6.iloc[:,:].values




In [ ]:
from tensorflow.keras.layers import Input,Dense,Reshape,Bidirectional,LSTM

In [ ]:
def text_only(text_arr, labels):

    input_layer1 = Input(shape=(text_arr.shape[1],))
    reshape_layer1 = Reshape((1, text_arr.shape[1]))(input_layer1)
    bilstm_output1 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer1)

    dense_layer = Dense(512, activation='relu')(bilstm_output1)
    dense_layer = Dense(256, activation='relu')(dense_layer)
    dense_layer = Dense(128, activation='relu')(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)

    output_layer = Dense(1, activation='sigmoid', name='class')(dense_layer)

    model1 = Model(inputs=input_layer1, outputs=output_layer)
    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model1.fit(text_arr[0:3926], labels[0:3926], epochs=10, batch_size=32)

    return model1

In [ ]:
def eval4(model1, text_arr, labels):
    # Reshape test data to match the expected input shape of the model
    text_test_data=text_arr[3926:]
    # img_test_data=img_arr[3926:]
    test_labels=labels[3926:]
    text_test_data_reshaped = text_test_data.reshape(-1, text_test_data.shape[1])
    # img_test_data_reshaped = img_test_data.reshape(-1, img_test_data.shape[1])

    prediction1=model1.predict(text_test_data)
    # prediction2 = model2.predict(text_test_data)
    # predictions=(prediction1+prediction2)/2.0
    binary_predictions = (prediction1 >= 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(test_labels, binary_predictions)
    precision = precision_score(test_labels, binary_predictions)
    recall = recall_score(test_labels, binary_predictions)
    f1 = f1_score(test_labels, binary_predictions)
    conf_matrix = confusion_matrix(test_labels, binary_predictions)

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("Confusion Matrix:\n", conf_matrix)



In [ ]:
def img_only(img_arr, labels):

    input_layer1 = Input(shape=(img_arr.shape[1],))
    reshape_layer1 = Reshape((1, img_arr.shape[1]))(input_layer1)
    bilstm_output1 = Bidirectional(LSTM(512, dropout=0.05, recurrent_dropout=0.2))(reshape_layer1)

    dense_layer = Dense(512, activation='relu')(bilstm_output1)
    dense_layer = Dense(256, activation='relu')(dense_layer)
    dense_layer = Dense(128, activation='relu')(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_layer)
    dense_layer = Dense(32, activation='relu')(dense_layer)
    dense_layer = Dense(16, activation='relu')(dense_layer)

    output_layer = Dense(1, activation='sigmoid', name='class')(dense_layer)

    model1 = Model(inputs=input_layer1, outputs=output_layer)
    model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model1.fit(img_arr[0:3926], labels[0:3926], epochs=10, batch_size=32)

    return model1

In [ ]:
def eval5(model1, img_arr, labels):
    # Reshape test data to match the expected input shape of the model
    # text_test_data=text_arr[3926:]
    img_test_data=img_arr[3926:]
    test_labels=labels[3926:]
    # text_test_data_reshaped = text_test_data.reshape(-1, text_test_data.shape[1])
    img_test_data_reshaped = img_test_data.reshape(-1, img_test_data.shape[1])

    prediction1=model1.predict(img_test_data)
    # prediction2 = model2.predict(text_test_data)
    # predictions=(prediction1+prediction2)/2.0
    binary_predictions = (prediction1 >= 0.5).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(test_labels, binary_predictions)
    precision = precision_score(test_labels, binary_predictions)
    recall = recall_score(test_labels, binary_predictions)
    f1 = f1_score(test_labels, binary_predictions)
    conf_matrix = confusion_matrix(test_labels, binary_predictions)

    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("Confusion Matrix:\n", conf_matrix)


In [ ]:
model=text_only(text_arr_final_df1,labels_final_df1)
print("Indic_bert")
eval4(model,text_arr_final_df1,labels_final_df1)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


model=text_only(text_arr_final_df2,labels_final_df2)
print("mbert")
eval4(model,text_arr_final_df2,labels_final_df2)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=text_only(text_arr_final_df3,labels_final_df3)
print("Indic_bart")
eval4(model,text_arr_final_df3,labels_final_df3)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")




Epoch 1/10
123/123 [==============================] - 9s 15ms/step - loss: 0.4858 - accuracy: 0.7557
Epoch 2/10
123/123 [==============================] - 3s 26ms/step - loss: 0.2742 - accuracy: 0.8895
Epoch 3/10
123/123 [==============================] - 2s 15ms/step - loss: 0.2404 - accuracy: 0.9014
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2252 - accuracy: 0.9124
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2050 - accuracy: 0.9170
Epoch 6/10
123/123 [==============================] - 2s 20ms/step - loss: 0.2034 - accuracy: 0.9205
Epoch 7/10
123/123 [==============================] - 3s 25ms/step - loss: 0.1904 - accuracy: 0.9256
Epoch 8/10
123/123 [==============================] - 5s 40ms/step - loss: 0.1814 - accuracy: 0.9294
Epoch 9/10
123/123 [==============================] - 4s 35ms/step - loss: 0.1772 - accuracy: 0.9312
Epoch 10/10
123/123 [==============================] - 4s 34ms/step - loss: 0.1792 - accura

Accuracy: 0.9331550802139037
Precision: 0.905
Recall: 0.9679144385026738
F1-score: 0.9354005167958656
Confusion Matrix:
 [[1176  133]
 [  42 1267]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 19s 39ms/step - loss: 0.4859 - accuracy: 0.7634
Epoch 2/10
123/123 [==============================] - 3s 24ms/step - loss: 0.2830 - accuracy: 0.8826
Epoch 3/10
123/123 [==============================] - 2s 14ms/step - loss: 0.2543 - accuracy: 0.8968
Epoch 4/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2655 - accuracy: 0.8882
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2205 - accuracy: 0.9106
Epoch 6/10
123/123 [==============================] - 3s 21ms/step - loss: 0.2082 - accuracy: 0.9152
Epoch 7/10
123/123 [==============================] - 3s 21

Accuracy: 0.8938120702826585
Precision: 0.8336569579288026
Recall: 0.983957219251337
F1-score: 0.9025928521373512
Confusion Matrix:
 [[1052  257]
 [  21 1288]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 9s 14ms/step - loss: 0.4905 - accuracy: 0.7578
Epoch 2/10
123/123 [==============================] - 2s 14ms/step - loss: 0.2989 - accuracy: 0.8726
Epoch 3/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2453 - accuracy: 0.8953
Epoch 4/10
123/123 [==============================] - 2s 14ms/step - loss: 0.2422 - accuracy: 0.9002
Epoch 5/10
123/123 [==============================] - 2s 14ms/step - loss: 0.2111 - accuracy: 0.9205
Epoch 6/10
123/123 [==============================] - 2s 19ms/step - loss: 0.1847 - accuracy: 0.9236
Epoch 7/10
123/123 [============================

In [ ]:
model=img_only(img_arr_final_df4,labels_final_df4)
print("resnet")
eval4(model,img_arr_final_df4,labels_final_df4)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=img_only(img_arr_final_df5,labels_final_df5)
print("vgg")
eval4(model,img_arr_final_df5,labels_final_df5)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")

model=img_only(img_arr_final_df6,labels_final_df6)
print("vit")
eval4(model,img_arr_final_df6,labels_final_df6)

print("--------------------------------------------------------------------------------------------")
print("--------------------------------------------------------------------------------------------")


Epoch 1/10
123/123 [==============================] - 10s 14ms/step - loss: 0.4101 - accuracy: 0.8174
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.1975 - accuracy: 0.9177
Epoch 3/10
123/123 [==============================] - 2s 17ms/step - loss: 0.0911 - accuracy: 0.9684
Epoch 4/10
123/123 [==============================] - 4s 31ms/step - loss: 0.0480 - accuracy: 0.9834
Epoch 5/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0330 - accuracy: 0.9906
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0282 - accuracy: 0.9901
Epoch 7/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0162 - accuracy: 0.9949
Epoch 8/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0189 - accuracy: 0.9926
Epoch 9/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0134 - accuracy: 0.9947
Epoch 10/10
123/123 [==============================] - 2s 14ms/step - loss: 0.0183 - accur

Accuracy: 0.865546218487395
Precision: 0.8906122448979592
Recall: 0.8334606569900688
F1-score: 0.861089187056038
Confusion Matrix:
 [[1175  134]
 [ 218 1091]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 8s 13ms/step - loss: 0.4669 - accuracy: 0.7776
Epoch 2/10
123/123 [==============================] - 2s 14ms/step - loss: 0.1587 - accuracy: 0.9379
Epoch 3/10
123/123 [==============================] - 2s 14ms/step - loss: 0.0728 - accuracy: 0.9778
Epoch 4/10
123/123 [==============================] - 3s 24ms/step - loss: 0.0496 - accuracy: 0.9834
Epoch 5/10
123/123 [==============================] - 2s 19ms/step - loss: 0.0260 - accuracy: 0.9908
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0306 - accuracy: 0.9898
Epoch 7/10
123/123 [=============================

Accuracy: 0.8059587471352178
Precision: 0.8211708099438653
Recall: 0.7822765469824293
F1-score: 0.8012519561815336
Confusion Matrix:
 [[1086  223]
 [ 285 1024]]
--------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------
Epoch 1/10
123/123 [==============================] - 8s 14ms/step - loss: 0.4247 - accuracy: 0.7995
Epoch 2/10
123/123 [==============================] - 2s 13ms/step - loss: 0.2863 - accuracy: 0.8762
Epoch 3/10
123/123 [==============================] - 3s 23ms/step - loss: 0.2000 - accuracy: 0.9200
Epoch 4/10
123/123 [==============================] - 2s 20ms/step - loss: 0.1246 - accuracy: 0.9529
Epoch 5/10
123/123 [==============================] - 2s 14ms/step - loss: 0.0727 - accuracy: 0.9771
Epoch 6/10
123/123 [==============================] - 2s 13ms/step - loss: 0.0587 - accuracy: 0.9778
Epoch 7/10
123/123 [===========================